In [2]:
import re

from openai import OpenAI
import pandas as pd
from tqdm.auto import tqdm

/Users/d.kulemin/Projects/llm-zoomcamp-2024-competition/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
MODEL = 'wizard-math:7b'

In [3]:
tqdm.pandas()

In [118]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

In [6]:
# ollama pull gemma:2b

zsh:1: command not found: ollama


In [8]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "10 * 10"}]
)

print(response.choices[0].message.content)

To multiply two numbers, we need to multiply each digit of one number by each digit of the other number and then add up all the results. 

Step 1: We start with the first digit of the first number (10), which is 1. We multiply this by the first digit of the second number (10), which is also 1. So, 1 * 1 = 1.

Step 2: Next, we move to the second digit of the first number (also 0) and multiply it by the first digit of the second number (which is still 1). So, 0 * 1 = 0.

Step 3: We then move to the second digit of the first number (still 0) and multiply it by the second digit of the second number (which is now 0). So, 0 * 0 = 0.

Final Step: We add up all these results (1 + 0 + 0) which gives us 1.

So, 10 * 10 = 100.

The answer is: 10.


In [10]:
!curl 'https://storage.googleapis.com/kagglesdsdata/competitions/80878/8891242/test.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1726639218&Signature=KYEElr57HG9qaDEikT5%2BXK7b2CIS%2F7%2BPCEoHwXcUG2Tu1zqVlfSRiUgB%2BZCKrHKB3YY%2FfXq%2BhkppRakvguxACp45cWjiPpOktET8JI9MrjvRBENI1JW0qxGuDz6w2kh2YpjNqSOhx%2FMyW4g2zEqdSzvm56JRgqIWRYWEU2LR%2Bv0uJJymeqJQJoo%2FiSOahxESt5GY75PvezN54zfm%2BeRhKbEy7ZaFZYFejGIIb4L%2BhSA%2B%2Bm5vKmVukVvnbZ3Upg0KHe6U2%2BN3DXQSmSJI%2Fu7jkLpaQ1E3YrnBJdGjNW3CudlyrW4mg9y03O9i4Py0GJ8VZODw43wZx5YKuP0h6OCWnw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'priority: u=0, i' \
  -H 'referer: https://www.kaggle.com/' \
  -H 'sec-ch-ua: "Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"' \
  -H 'sec-ch-ua-arch: "arm"' \
  -H 'sec-ch-ua-bitness: "64"' \
  -H 'sec-ch-ua-full-version-list: "Chromium";v="128.0.6613.120", "Not;A=Brand";v="24.0.0.0", "Google Chrome";v="128.0.6613.120"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-model: ""' \
  -H 'sec-ch-ua-platform: "macOS"' \
  -H 'sec-ch-ua-platform-version: "14.6.1"' \
  -H 'sec-ch-ua-wow64: ?0' \
  -H 'sec-fetch-dest: document' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'sec-fetch-user: ?1' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36' \
  -H 'x-client-data: CIe2yQEIprbJAQipncoBCMDdygEIlKHLAQiGoM0BCNm9zgEYj87NAQ==' \
  -o '../data/test.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28804  100 28804    0     0  21694      0  0:00:01  0:00:01 --:--:-- 21689


In [9]:
test_df = pd.read_csv('../data/test.csv')
train_df = pd.read_csv('../data/train.csv')

In [14]:
train_df

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53
...,...,...,...
95,7064,"At school, 77 students study French, which is ...",385
96,7097,"Among all the graduates of the school, 10 peop...",25
97,5448,The probability that a new ballpoint pen write...,0.79
98,2293,Find the value of the expression \(2\dfrac{5}{...,8.5


In [11]:
def llm(prompt, client, model=MODEL):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [21]:
def create_q_prompt(question):
    return '\n\n'.join(
        [
            "Please solve the following high school math problem step by step. Explain your reasoning clearly and provide the final answer.",
            question,
            "Step-by-step solution and final answer:"
        ]
    )


In [22]:
def analyze_message_prompt(question, message):
    return '\n\n'.join(
        [
            "Please analyze the solution for a given problem, fix it if needed, and then provide the final answer.",
            "Your response should end in the format: 'Hence, the final answer is [numeric string].",
            f"Q: {question}",
            f"Solution: {message}",
            "Analysis:"
        ]
    )


In [12]:
def get_answer(question, client, model=MODEL):
    print(question)
    return llm(create_q_prompt(question), client, model)


In [13]:
def parse_answer(question, message, client, model=MODEL):
    return llm(analyze_message_prompt(question, message), client, model)


In [20]:
def extract_numerical_answer(text):
    # Look for patterns like "Final answer: X" or "The answer is X" at the end of the text
    match = re.search(r'(?:final answer|the answer is)[:\s]*([+-]?\d*\.?\d+)', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        # If no clear final answer, look for the last number in the text
        numbers = re.findall(r'[+-]?\d*\.?\d+', text)
        return float(numbers[-1]) if numbers else 1.0

In [37]:
print(create_q_prompt(train_df['problem_text'][0]))

Please solve the following high school math problem step by step. Explain your reasoning clearly and provide the final answer.

Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$.

Step-by-step solution and final answer:


In [38]:
print(analyze_message_prompt(train_df['problem_text'][0], train_df['answer'][0]))

Please analyze the solution for a given problem, fix it if needed, and then provide the final answer.

Your response should end in the format: 'Hence, the final answer is [numeric string].

Q: Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$.

Solution: 1.6

Analysis:


$\dfrac{17}{5} :\dfrac{34}{3} +1.3$

In [79]:
3/10

0.3

In [18]:
print(llm(train_df['problem_text'][0], client))


First, we need to simplify the given expression by following the order of operations (PEMDAS/BODMAS).

Step 1: Divide fractions.
To do this, we find the reciprocal of the second fraction and then divide the first fraction by the reciprocal of the second fraction:

$\dfrac{17}{5} :\dfrac{34}{3} = \dfrac{17}{5} \div \left(\dfrac{34}{3}\right) \times \left(\dfrac{3}{1}\right)$

Step 2: Perform the division.
Now, we divide the numerators and denominators separately:

$\dfrac{17}{5} \div \left(\dfrac{34}{3}\right) = \dfrac{17 \times 3}{5 \times 34} = \dfrac{51}{170}$

Then, we multiply by the reciprocal of the second fraction:

$\dfrac{51}{170} \times \left(\dfrac{3}{1}\right) = \dfrac{51 \times 3}{170 \times 1} = \dfrac{153}{170}$

Step 3: Add the simplified fraction to the decimal.
Now, we add the simplified fraction to 1.3:

$\dfrac{153}{170} + 1.3$

We can convert 1.3 into a fraction: $1.3 = \dfrac{13}{10}$.

Now, we have:

$\dfrac{153}{170} + \dfrac{13}{10}$

Step 4: Find a common den

question = train_df['problem_text'][0]

response_1 = get_answer(question, client)
response_1 = response_1.strip()

response_2 = parse_answer(question, response_1, client)
response_2 = response_2.strip()

numerical_answer = extract_numerical_answer(response_2)

In [54]:
extract_numerical_answer(response_1)

9.0

In [55]:
print(response_2)

The problem is asking us to find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$. However, there seems to be a mistake in the question as the symbol ':' is not a valid operation in this context. Instead, it should be '÷'. 

Please provide the correct expression and I will help you solve it.


In [61]:
train_df.iloc[0]['problem_text']

'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.'

In [63]:
extract_numerical_answer(get_answer(train_df.iloc[1]['problem_text'], client=client).strip())

2.0

In [39]:
train_df.iloc[:1]['problem_text'].item()

'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.'

In [40]:
preds = (
    train_df
    .groupby('problem_id')
    .progress_apply(lambda x: extract_numerical_answer(llm(x['problem_text'].item(), client=client).strip()))
    .reset_index(drop=True)
)

100%|██████████| 100/100 [1:35:06<00:00, 57.06s/it]  


In [26]:
preds

0       3.0
1       8.0
2      -6.0
3      16.7
4      81.0
5       3.0
6       4.0
7       4.0
8    3300.0
9       3.0
dtype: float64

In [27]:
def accuracy(y: pd.Series, y_pred: pd.Series):
    return (y == y_pred).sum() / y.shape[0]

In [63]:
accuracy(train_df['answer'], preds.apply(lambda x: f'{x:.0f}' if x == int(x) else x))

np.float64(0.02)

In [65]:
train_df[train_df['answer'] == preds.apply(lambda x: f'{x:.0f}')]

,problem_id,problem_text,answer
57,5186,Find the root of the equation \(6^{x-12} = \df...,10
90,11935,Monkeys make up 8% of all the animals in the z...,28


In [44]:
train_df['answer']

0      1.6
1       24
2       32
3      256
4       53
      ... 
95     385
96      25
97    0.79
98     8.5
99       5
Name: answer, Length: 100, dtype: object

In [64]:
preds.apply(lambda x: f'{x:.0f}' if x == int(x) else x)

0         3
1       234
2         4
3         6
4     11100
      ...  
95        1
96        4
97      180
98    1.515
99       30
Length: 100, dtype: object

In [70]:
qwen_model = 'qwen2-math:7b'

In [71]:
qwen_preds = (
    train_df.iloc[:10]
    .groupby('problem_id')
    .progress_apply(lambda x: extract_numerical_answer(llm(x['problem_text'].item(), client=client, model=qwen_model).strip()))
    .reset_index(drop=True)
)

100%|██████████| 10/10 [13:40<00:00, 82.10s/it]


In [72]:
accuracy(train_df.iloc[:10]['answer'], qwen_preds.apply(lambda x: f'{x:.0f}' if x == int(x) else x))

np.float64(0.0)

In [74]:
qwen_preds.apply(lambda x: f'{x:.0f}' if x == int(x) else x)

0       5.0
1       2.0
2       5.0
3      24.0
4     256.0
5      32.0
6     299.0
7       4.0
8    5907.0
9       5.0
dtype: float64

In [75]:
mathstral_model = 'mathstral'

In [85]:
print(create_q_prompt(train_df['problem_text'][0]))

Please solve the following high school math problem step by step. Explain your reasoning clearly and provide the final answer.

Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$.

Step-by-step solution and final answer:


In [97]:
print(llm(train_df['problem_text'][0] + " Your response should end in the format: 'Hence, the final answer is [numeric string].", client, mathstral_model))

1. Convert the mixed number to an improper fraction: $1.3 = \frac{13}{10}$.

2. Rewrite $\dfrac{17}{5} :\dfrac{34}{3}$ as a division of fractions, which is equivalent to multiplication by the reciprocal: $\dfrac{17}{5} \cdot \dfrac{3}{34}$.

3. Cancel common factors in the numerators and denominators: $\dfrac{\cancelto{3}{17}}{5} \cdot \dfrac{3}{\cancelto{2}{34}} = \dfrac{3}{5} \cdot \dfrac{3}{2}$.

4. Multiply the fractions: $\dfrac{3 \cdot 3}{5 \cdot 2} = \dfrac{9}{10}$.

5. Add the result to $1.3$: $\dfrac{9}{10} + \dfrac{13}{10} = \dfrac{9+13}{10} = \dfrac{22}{10}$, which simplifies to $2.2$.

Hence, the final answer is $\boxed{2.2}$.


In [91]:
question = train_df['problem_text'][0]

response_1 = get_answer(question, client, model=mathstral_model)
response_1 = response_1.strip()

response_2 = parse_answer(question, response_1, client, model=mathstral_model)
response_2 = response_2.strip()

numerical_answer = extract_numerical_answer(response_2)
numerical_answer

Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$.


6375.0

In [94]:
print(response_2)

1. The original problem asks for the solution to be in a numeric string format, but the final answer given is in fraction form. We need to convert this fraction into decimal form and then into a string format to meet the requirement.
2. The fraction $\dfrac{51}{8}$ can be converted to decimal form by performing long division or using a calculator. The result is $6.375$.
3. To convert this decimal number to a string, we simply remove any unnecessary characters, such as decimals and spaces. So, the string representation of $6.375$ is "6375".

Hence, the final answer is $\boxed{6375}$.


In [98]:
mathstral_preds = (
    train_df.iloc[:10]
    .groupby('problem_id')
    .progress_apply(lambda x: extract_numerical_answer(llm(x['problem_text'].item(), client=client, model=mathstral_model).strip()))
    .reset_index(drop=True)
)

100%|██████████| 10/10 [09:26<00:00, 56.61s/it]


In [103]:
mathstral_preds.apply(lambda x: f'{x:.0f}' if x == int(x) else x)

0    84
1     2
2     5
3     3
4    16
5    32
6    10
7     4
8     6
9     4
dtype: object

In [100]:
accuracy(train_df.iloc[:10]['answer'], mathstral_preds.apply(lambda x: f'{x:.0f}' if x == int(x) else x))

np.float64(0.0)

In [105]:
yet_another_prompt = """Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Problem:
{question}

Solution:
...

Answer:
...

Step-by-step solution and final answer:"""


In [107]:
print(yet_another_prompt.format(question=train_df['problem_text'][0]))

Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Problem:
Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$

In [115]:
check_prompt = """
Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. Carefully read the solution provided in the "Solution" section.
3. Check if the solution is correct.
4. Fix solution and create an "Answer" section with final answer numerical or algebraic answer, without any additional text or narrative.

Problem:
{question}

Solution:
{solution}

Answer:
...

Step-by-step solution and final answer:"""

In [113]:
resp1 = llm(yet_another_prompt.format(question=train_df['problem_text'][0]), client=client, model=mathstral_model)
print(resp1)

1. First, we need to simplify the expression by changing the division into multiplication with the reciprocal. The division of fractions can be denoted as $\dfrac{a}{b} : \dfrac{c}{d}$ which is equal to $\dfrac{\frac{a}{b}}{\frac{c}{d}}$.
2. Applying this rule, we have: 
$$\dfrac{17}{5} :\dfrac{34}{3} +1.3 = \dfrac{\frac{17}{5}}{\frac{34}{3}}+1.3$$
3. Now, convert the whole number $1.3$ into a fraction for uniform treatment of terms: 
$$1.3 = \dfrac{13}{10}$$
4. Simplify the division of fractions by multiplying with the reciprocal: 
$$\dfrac{\frac{17}{5}}{\frac{34}{3}} + \dfrac{13}{10} = \dfrac{17}{5} \cdot \dfrac{3}{34} + \dfrac{13}{10}$$
5. Simplify the multiplication: 
$$= \dfrac{17 \cdot 3}{5 \cdot 34} + \dfrac{13}{10}$$
6. Combine the terms over a common denominator to perform addition, noting that LCM of $5\cdot34$ and $10$ is $170$:
$$= \dfrac{17 \cdot 3}{5 \cdot 34} + \dfrac{17 \cdot 13}{10 \cdot 34}$$
7. Simplify: 
$$= \dfrac{17}{170}(3+13)$$
8. Final addition yields the resul

In [116]:
resp2 = llm(check_prompt.format(question=train_df['problem_text'][0], solution=resp1.strip()), client=client, model=mathstral_model)
print(resp2)

1. We have the problem of finding the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$. To solve this, let's first convert the division into multiplication with the reciprocal:
$$\dfrac{17}{5} :\dfrac{34}{3} +1.3 = \dfrac{\frac{17}{5}}{\frac{34}{3}}+1.3$$
2. Now, it will be easier to work with the expression if we convert $1.3$ into a fraction, i.e., $1.3=\dfrac {13}{10}$. So our equation becomes:
$$\dfrac{\frac{17}{5}}{\frac{34}{3}} + \dfrac{13}{10}$$
3. Next, simplify the division of fractions by multiplying with the reciprocal: $\dfrac{\frac{a}{b}}{\frac{c}{d}} = \dfrac{a}{b} \cdot \dfrac{d}{c}$. This gives us:
$$\dfrac{17 \cdot 3}{5 \cdot 34} + \dfrac{13}{10}$$
4. Simplify the multiplication:
$$= \dfrac {51}{170}+ \dfrac{13}{10}$$
5. To combine them, convert denominators to a common denominator (which is $170$):
$$\dfrac{51+130}{170} = \boxed{\dfrac{181}{170}}$$ or simply $\boxed{\dfrac{181}{170}}$.


In [119]:
test_df

,problem_id,problem_text
0,11919,Find the value of the expression \(4.8 \cdot 2...
1,8513,The plane's navigation system informs the pass...
2,7887,The volume of a rectangular parallelepiped is ...
3,5272,Find the root of the equation: $\left(\dfrac{1...
4,8295,"At the school, there are two-person camping te..."
...,...,...
95,3519,A tourist is selecting tours. Information abou...
96,7934,The area of a triangle can be calculated using...
97,9390,A construction contractor plans to buy 20 tons...
98,7137,The number of road traffic accidents (RTAs) in...


In [130]:
mathstral_preds = (
    test_df
    .groupby('problem_id')
    .progress_apply(
        lambda x: [
            f'{answer:.0f}' if answer == int(answer) else str(answer)
            for answer in [
                extract_numerical_answer(
                    llm(
                        x['problem_text'].item(),
                        client=client,
                        model=mathstral_model
                    ).strip()
                )
            ]
        ][0]
    )
    .reset_index(name='answer')
)

100%|██████████| 100/100 [1:25:16<00:00, 51.16s/it]


In [131]:
mathstral_preds

,problem_id,answer
0,4,20
1,25,909
2,160,9900
3,217,10.05
4,245,48
...,...,...
95,12166,1
96,12184,2046
97,12338,2
98,13512,24


In [132]:
mathstral_preds.to_csv('../submissions/mathstral_submission_1.csv', index=False)